In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

In [2]:
tourney_result = pd.read_csv('../input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MNCAATourneyCompactResults.csv')



In [3]:
#Cleaning Up Tourney Results

tourney_result = tourney_result.drop(['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'],axis=1)
tourney_result

,Season,WTeamID,LTeamID
0,1985,1116,1234
1,1985,1120,1345
2,1985,1207,1250
3,1985,1229,1425
4,1985,1242,1325
5,1985,1246,1449
6,1985,1256,1338
7,1985,1260,1233
8,1985,1314,1292
9,1985,1323,1333


In [4]:
#Reading KenPom Values
import os, glob

path = '../input/kenpom/'

all_kenpom = glob.glob(os.path.join(path, "*.csv"))

all_df = []
for f in all_kenpom:
    df = pd.read_csv(f, sep=',')
    df['Season'] = '20' + f.split('/kenpom\summary')[-1]
    all_df.append(df)
    
merged_df = pd.concat(all_df, ignore_index=True, sort=True)

merged_df['Season'] = merged_df['Season'].str.slice(0,4,1)



kenpom_data = merged_df[['AdjEM','AdjOE','AdjDE','AdjTempo','Season', 'TeamName']]

kenpom_data

,AdjEM,AdjOE,AdjDE,AdjTempo,Season,TeamName
0,34.022000,121.3740,87.3522,73.9016,2002,Duke
1,30.844200,117.4480,86.6033,66.9485,2002,Cincinnati
2,27.788300,118.1000,90.3113,77.6019,2002,Kansas
3,26.853400,118.1510,91.2974,73.3263,2002,Maryland
4,25.717600,115.3870,89.6694,69.5612,2002,Florida
5,24.863700,113.3060,88.4422,65.8584,2002,Oklahoma
6,24.123600,119.9240,95.8006,70.5594,2002,Oregon
7,22.921300,112.5950,89.6736,64.7779,2002,Indiana
8,22.261500,113.2980,91.0361,64.5763,2002,Marquette
9,22.179200,112.5360,90.3566,67.6361,2002,Kentucky


In [5]:
team_names = pd.read_csv('../input/TeamSpellings.csv')

#all_names = pd.read_csv('../input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MTeams.csv')

#all_names = all_names.astype(str)

kenpom_data['TeamName'] = kenpom_data['TeamName'].str.lower()

kenpom_data = kenpom_data.merge(team_names, on='TeamName',how='left')

#kenpom_data.to_csv('test2.csv')

kenpom_data = kenpom_data.drop(['TeamName'],axis=1)





C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [6]:
df1 = tourney_result
df2 = kenpom_data

df1['Season']=df1['Season'].astype(int)
df1['WTeamID']=df1['WTeamID'].astype(int)
df1['LTeamID']=df1['LTeamID'].astype(int)

df2['Season']=df2['Season'].astype(int)
df2['TeamID']=df2['TeamID'].astype(int)


#tourney_result = pd.merge(tourney_result, kenpom_data, left_on=['Season','WTeamID'], right_on=['Season','TeamID'], how='left')

#inner join drops years 1985-2001 where KenPom data is not available

df1 = df1.merge(df2,left_on=['Season','WTeamID'],right_on=['Season','TeamID'], how="inner")
df1.rename(columns={'AdjOE':'WAdjOE','AdjDE':'WAdjDE','AdjEM':'WAdjEM','AdjTempo':'WAdjTempo'},inplace=True)
df1 = df1.drop(['TeamID'],axis=1)

df1 = df1.merge(df2,left_on=['Season','LTeamID'],right_on=['Season','TeamID'], how="inner")
df1.rename(columns={'AdjOE':'LAdjOE','AdjDE':'LAdjDE','AdjEM':'LAdjEM','AdjTempo':'LAdjTempo'},inplace=True)
df1 = df1.drop(['TeamID'],axis=1)

indexNames = df1[df1['Season']>2014].index

df1.drop(indexNames,inplace=True)

df1

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
df_win = df1.drop(['Season','WTeamID','LTeamID','WAdjOE','WAdjDE','LAdjOE','LAdjDE'],axis=1)
df_win.rename(columns={'WAdjEM':'AdjEM1', 'LAdjEM':'AdjEM2', 'WAdjTempo':'AdjTempo1', 'LAdjTempo':'AdjTempo2'}, inplace=True)
df_win

In [ ]:
df_lose = df_win.copy()
df_lose['AdjEM1'] = df_win['AdjEM2']
df_lose['AdjEM2'] = df_win['AdjEM1']
df_lose['AdjTempo1'] = df_win['AdjTempo2']
df_lose['AdjTempo2'] = df_win['AdjTempo1']
df_lose

In [ ]:
df_win['EM_diff'] = df_win['AdjEM1'] - df_win['AdjEM2']
df_lose['EM_diff'] = df_lose['AdjEM1'] - df_lose['AdjEM2']

df_win['result'] = 1
df_lose['result'] = 0

df_final = pd.concat((df_win,df_lose)).reset_index(drop=True)

#df_final

df_final = df_final.drop(['AdjTempo1','AdjTempo2','AdjEM1','AdjEM2'],axis=1)

In [ ]:
feature_cols=['EM_diff']
X = df_final[feature_cols]
y= df_final.result

#X_train = df_final.drop('result',axis=1)
#y_train = df_final.result
#X_train, y_train = shuffle(X_train, y_train)


In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train,y_train)
y_pred=logreg.predict(X_test)



#logreg = LogisticRegression()
#params = {'C':np.logspace(start=-5,stop=3,num=9)}
#clf = GridSearchCV(logreg,params,scoring='neg_log_loss',refit=True)
#clf.fit(X_train, y_train)
#print('Best log_loss: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_['C']))

In [ ]:
from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_test,y_pred)
cnf_matrix


In [ ]:
print('Accuracy:',metrics.accuracy_score(y_test, y_pred))
print('Precision:',metrics.precision_score(y_test, y_pred))
print('Recall:',metrics.recall_score(y_test, y_pred))
print('Log-Loss:',metrics.log_loss(y_test,y_pred))



In [ ]:
y_pred_proba = logreg.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label='data 1, auc='+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
test_df = pd.read_csv('../input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MSampleSubmissionStage1_2020.csv')
test_df['Season'] = test_df['ID'].map(lambda x: int(x[:4]))
test_df['WTeamID'] = test_df['ID'].map(lambda x: int(x[5:9]))
test_df['LTeamID'] = test_df['ID'].map(lambda x: int(x[10:14]))
test_df

In [ ]:
kenpom_data
kenpom_datav2 = kenpom_data.drop(['AdjOE','AdjDE'],axis=1)

test_df = test_df.merge(kenpom_datav2, left_on=['Season','WTeamID'],right_on=['Season','TeamID'],how='left')
test_df.rename(columns={'AdjEM':'AdjEM1','AdjTempo':'AdjTempo1'},inplace=True)
test_df = test_df.drop('TeamID',axis=1)

test_df = test_df.merge(kenpom_datav2, left_on=['Season','LTeamID'],right_on=['Season','TeamID'],how='left')
test_df.rename(columns={'AdjEM':'AdjEM2','AdjTempo':'AdjTempo2'},inplace=True)
test_df = test_df.drop('TeamID',axis=1)

test_df


In [ ]:
test_df['EM_diff'] = test_df['AdjEM1'] - test_df['AdjEM2']
test_df = test_df.drop(['ID','Pred','Season','WTeamID','LTeamID','AdjTempo1','AdjTempo2','AdjEM1','AdjEM2'],axis=1)
test_df

In [ ]:
#test_df['AdjEM1'] = float(test_df['AdjEM1'])
#test_df['AdjTempo1'] = float(test_df['AdjTempo1'])
#test_df['AdjEM2'] = float(test_df['AdjEM2'])
#test_df['AdjTempo2'] = float(test_df['AdjTempo2'])
#test_df['EM_diff'] = float(test_df['EM_diff'])

#pd.isnull(test_df).sum()>0

#np.where(np.isnan(test_df))

#test_df.to_csv('testing.csv')

test_preds = logreg.predict_proba(test_df)[:, 1]
test_preds

In [ ]:
submission_df = pd.read_csv('../input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MSampleSubmissionStage1_2020.csv')
submission_df['Pred'] = test_preds
submission_df

In [ ]:
submission_df['Pred'].hist()

In [ ]:
submission_df.to_csv('../submissions/submission_v2.csv',index=False)